In [25]:
import pandas as pd
import numpy as np

# # 출력할 최대 열 수 설정
# pd.set_option('display.max_columns', None)

# # 출력할 최대 행 수 설정
# pd.set_option('display.max_rows', None)

import warnings

# 모든 경고 무시
warnings.filterwarnings("ignore")

## 관광DB데이터

In [26]:
df = pd.read_csv('../data/tour_api/tourapi_basic_info.csv', encoding = 'utf-8-sig')
df.head()

,addr1,addr2,areacode,booktour,cat1,cat2,cat3,contentid,contenttypeid,createdtime,...,firstimage2,cpyrhtDivCd,mapx,mapy,mlevel,modifiedtime,sigungucode,tel,title,zipcode
0,전라남도 신안군 흑산면 가거도길 38-2,(흑산면),38.0,0.0,A01,A0101,A01011300,127480,12,20030905090000,...,NaN,NaN,125.112515,34.074017,6.0,20230918102221,12.0,NaN,가거도(소흑산도),58866
1,전라남도 진도군 고군면 신비의바닷길 47,(고군면),38.0,0.0,A01,A0101,A01011200,126273,12,20031107090000,...,http://tong.visitkorea.or.kr/cms/resource/36/3...,Type3,126.354741,34.435459,6.0,20240110114813,21.0,NaN,가계해수욕장,58911
2,경상남도 창원시 마산합포구 성호서7길 15-8,NaN,36.0,0.0,A02,A0203,A02030100,2019720,12,20150721030848,...,http://tong.visitkorea.or.kr/cms/resource/55/2...,Type3,128.569655,35.207766,6.0,20230412105954,16.0,NaN,가고파 꼬부랑길 벽화마을,51281
3,강원특별자치도 삼척시 가곡면 탕곡리,509-3,32.0,NaN,A02,A0202,A02020300,2994116,12,20230717155822,...,http://tong.visitkorea.or.kr/cms/resource/54/2...,Type3,129.206230,37.150749,6.0,20230807093649,4.0,033-572-1800,가곡유황온천&스파,25954
4,경기도 양주시 장흥면 권율로 117,NaN,31.0,0.0,A02,A0202,A02020600,129194,12,20060807090000,...,http://tong.visitkorea.or.kr/cms/resource/46/2...,Type3,126.949750,37.725452,6.0,20231213141137,18.0,NaN,가나아트파크,11520


In [27]:
# 지역명 매핑을 위한 사전
region_mapping = {
    '경기': '경기도', '서울시': '서울특별시', '서울': '서울특별시', '인천': '인천광역시', '부산': '부산광역시',
    '대구': '대구광역시', '울산': '울산광역시', '대전': '대전광역시', '광주': '광주광역시', '제주': '제주특별자치도',
    '제주도': '제주특별자치도', '충남': '충청남도', '충북': '충청북도', '전남': '전라남도', '전북': '전라북도',
    '경남': '경상남도', '경북': '경상북도', '세종': '세종특별자치시', '세종시': '세종특별자치시',  '\u200b부산': '부산광역시', 
    '\u200b강원도': '강원도',  '대전광역시유성구': '대전광역시'}

# 'addr1' 열의 데이터를 표준 지역명으로 매핑 후 재정리
df['standardized_region'] = df['addr1'].apply(lambda x: region_mapping.get(str(x).split(' ')[0], str(x).split(' ')[0]) if pd.notna(x) else np.nan)

# 표준화된 지역명에서 유니크한 값 추출
unique_standardized_regions = df['standardized_region'].unique()

unique_standardized_regions

array(['전라남도', '경상남도', '강원특별자치도', '경기도', '부산광역시', '경상북도', '전북특별자치도',
       '제주특별자치도', '충청북도', '충청남도', '대구광역시', '서울특별시', '울산광역시', '대전광역시',
       '인천광역시', '거창군', '강원도', '광주광역시', '세종특별자치시', '고양시', '수원시', '전라북도',
       nan, '경주시', '서울시강동구', '청도군', '청주시', '포항시', '태백시', '제주시', '전주시',
       '대구시', '온라인개최', '통영시', '대부황금로1504', '인천시', '강릉시', '119-11,',
       '13-29,', '강원', '18,', '부산시', '광주시', '울산시', '속초시', '세종특별시',
       '한식\t서울특별시', '춘천시', '양주시'], dtype=object)

In [28]:
df['contenttypeid'].value_counts()

contenttypeid
39    16572
12    13050
38     9259
28     4714
32     3600
14     2663
15     1410
25     1070
Name: count, dtype: int64

- 관광타입(12:관광지, 14:문화시설, 15:축제공연행사, 25:여행코스, 28:레포츠, 32:숙박, 38:쇼핑, 39:음식점) 을 고려해 불필요 정보 필터링
- 관광지, 문화시설, 여행코스 만 선정

In [29]:
# contenttypeid 값이 12, 14, 28인 행만 필터링
travel_df = df[df['contenttypeid'].isin([12, 14, 28])]

selected_columns = ['contentid', 'contenttypeid', 'title', 'addr1', 'addr2', 'areacode', 
                    'sigungucode', 'zipcode', 'cat1', 'cat2', 'cat3', 'mapx', 'mapy']

# 선택된 컬럼만을 포함하고 순서대로 정렬한 새 DataFrame 생성
travel_df = travel_df[selected_columns]

# 필터링된 데이터 프레임을 확인합니다.
travel_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20427 entries, 0 to 22906
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   contentid      20427 non-null  int64  
 1   contenttypeid  20427 non-null  int64  
 2   title          20427 non-null  object 
 3   addr1          20412 non-null  object 
 4   addr2          6233 non-null   object 
 5   areacode       20421 non-null  float64
 6   sigungucode    20418 non-null  float64
 7   zipcode        19730 non-null  object 
 8   cat1           20427 non-null  object 
 9   cat2           20427 non-null  object 
 10  cat3           20427 non-null  object 
 11  mapx           20427 non-null  float64
 12  mapy           20427 non-null  float64
dtypes: float64(4), int64(2), object(7)
memory usage: 2.2+ MB


In [30]:
travel_df.head()

,contentid,contenttypeid,title,addr1,addr2,areacode,sigungucode,zipcode,cat1,cat2,cat3,mapx,mapy
0,127480,12,가거도(소흑산도),전라남도 신안군 흑산면 가거도길 38-2,(흑산면),38.0,12.0,58866,A01,A0101,A01011300,125.112515,34.074017
1,126273,12,가계해수욕장,전라남도 진도군 고군면 신비의바닷길 47,(고군면),38.0,21.0,58911,A01,A0101,A01011200,126.354741,34.435459
2,2019720,12,가고파 꼬부랑길 벽화마을,경상남도 창원시 마산합포구 성호서7길 15-8,NaN,36.0,16.0,51281,A02,A0203,A02030100,128.569655,35.207766
3,2994116,12,가곡유황온천&스파,강원특별자치도 삼척시 가곡면 탕곡리,509-3,32.0,4.0,25954,A02,A0202,A02020300,129.206230,37.150749
4,129194,12,가나아트파크,경기도 양주시 장흥면 권율로 117,NaN,31.0,18.0,11520,A02,A0202,A02020600,126.949750,37.725452


In [46]:
travel_df[travel_df['title'].str.contains('에코랜드')]

,contentid,contenttypeid,title,addr1,addr2,areacode,sigungucode,zipcode,cat1,cat2,cat3,mapx,mapy
1313,2500326,12,구미에코랜드,경상북도 구미시 산동읍 인덕1길 195,NaN,35.0,4.0,39159,A02,A0204,A02040800,128.462488,36.177155
7491,3007648,12,에코랜드,경기도 남양주시 별내면 광전리,NaN,31.0,9.0,12090,A02,A0202,A02020700,127.127517,37.703594
7492,1146121,12,에코랜드테마파크,제주특별자치도 제주시 조천읍 번영로 1278-169,NaN,39.0,4.0,63346,A02,A0202,A02020600,126.667062,33.456487


In [32]:
travel_df[travel_df['addr1'].str.contains('경기도', na = False)]

,contentid,contenttypeid,title,addr1,addr2,areacode,sigungucode,zipcode,cat1,cat2,cat3,mapx,mapy
4,129194,12,가나아트파크,경기도 양주시 장흥면 권율로 117,NaN,31.0,18.0,11520,A02,A0202,A02020600,126.949750,37.725452
5,2777865,12,가남체육공원,경기도 여주시 가남읍 대명산길 98,NaN,31.0,20.0,12662,A02,A0202,A02020700,127.534914,37.201709
16,1553085,12,가루매마을,경기도 양평군 지평면 부일길 101,(지평면),31.0,19.0,12544,A02,A0203,A02030100,127.614019,37.451735
17,125463,12,가리산(포천),경기도 포천시 이동면 장암리,NaN,31.0,29.0,11111,A01,A0101,A01010400,127.402805,38.043301
26,2745573,12,가막들공원,경기도 의왕시 양지편로 41-2,(청계동),31.0,24.0,16007,A02,A0202,A02020700,126.995697,37.391692
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22829,1197248,28,[북한산 둘레길] 21 우이령길,경기도 양주시 장흥면 교현리,NaN,31.0,18.0,11523,A03,A0302,A03022700,126.970251,37.699930
22852,1964927,28,[서울둘레길 6코스] 안양천코스,경기도 안양시 만안구 경수대로 1431,(석수동),31.0,17.0,13906,A03,A0302,A03022700,126.902543,37.434396
22855,1030084,28,[시흥 늠내길 제1코스] 숲길,경기도 시흥시 시청로,(장현동),31.0,14.0,14998,A03,A0302,A03022700,126.802989,37.380131
22856,1030109,28,[시흥 늠내길 제2코스] 갯골길,경기도 시흥시 장곡동,NaN,31.0,14.0,14971,A03,A0302,A03022700,126.802628,37.379062


## 여행로그DB

In [33]:
travel_log = pd.read_csv('../data/travel_log/travel_log.csv', encoding='utf-8-sig')

travel_log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57909 entries, 0 to 57908
Data columns (total 33 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   TRAVEL_ID             57909 non-null  object 
 1   TRAVEL_NM             52031 non-null  object 
 2   TRAVELER_ID           52031 non-null  object 
 3   VISIT_AREA_ID         57909 non-null  int64  
 4   VISIT_AREA_NM         57909 non-null  object 
 5   VISIT_AREA_TYPE_CD    57909 non-null  int64  
 6   TRAVEL_YMD            57909 non-null  object 
 7   POI_ID                26571 non-null  object 
 8   POI_NM                26571 non-null  object 
 9   X_COORD               55290 non-null  object 
 10  Y_COORD               55290 non-null  object 
 11  SGG_CD                4370 non-null   float64
 12  ROAD_NM_CD            0 non-null      float64
 13  ROAD_NM_ADDR          37327 non-null  object 
 14  LOTNO_CD              974 non-null    object 
 15  LOTNO_ADDR         

In [42]:
# 선택하고 싶은 컬럼 목록
selected_columns = ['POI_ID', 'VISIT_AREA_TYPE_CD','VISIT_AREA_NM', 'ROAD_NM_ADDR', 'LOTNO_ADDR', 'TRAVEL_ID', 'TRAVEL_NM', 'TRAVELER_ID', 'VISIT_AREA_ID',
                    'POI_ID', 'POI_NM', 'TRAVEL_YMD',
                    'STARS']

# 선택된 컬럼만을 포함하는 새로운 DataFrame 생성
filtered_travel_log = travel_log[selected_columns]

In [43]:
filtered_travel_log.head()

,POI_ID,VISIT_AREA_TYPE_CD,VISIT_AREA_NM,ROAD_NM_ADDR,LOTNO_ADDR,TRAVEL_ID,TRAVEL_NM,TRAVELER_ID,VISIT_AREA_ID,POI_ID,POI_NM,TRAVEL_YMD,STARS
0,NaN,7,프로방스마을,경기 파주시 탄현면 새오리로 77,경기 파주시 탄현면 성동리 82-1,a_a015688,A03,a015688,2210300006,NaN,NaN,2022-10-30,4.0
1,NaN,4,더현대서울,서울 영등포구 여의대로 108,서울 영등포구 여의도동 22,a_a000491,A01,a000491,2208200003,NaN,NaN,2022-08-20,5.0
2,POI01000000DNNN06,4,강릉중앙시장,강원 강릉시 금성로 21,강원 강릉시 성남동 50,a_a000172,A01,a000172,2208110007,POI01000000DNNN06,신흥수산,2022-08-11,4.0
3,NaN,7,청계천,NaN,서울 종로구 서린동 148,a_a000554,A01,a000554,2208270003,NaN,NaN,2022-08-27,4.0
4,NaN,7,삼송역 3호선,경기 고양시 덕양구 삼송로 지하 194,경기 고양시 덕양구 삼송동 18-5,a_a005195,A03,a005195,2210140002,NaN,NaN,2022-10-14,3.0


In [44]:
filtered_travel_log[filtered_travel_log['VISIT_AREA_NM'].str.contains('에코랜드')].head(20)

,POI_ID,VISIT_AREA_TYPE_CD,VISIT_AREA_NM,ROAD_NM_ADDR,LOTNO_ADDR,TRAVEL_ID,TRAVEL_NM,TRAVELER_ID,VISIT_AREA_ID,POI_ID,POI_NM,TRAVEL_YMD,STARS
14787,NaN,13,구미에코랜드,경북 구미시 산동읍 인덕1길 195,경북 구미시 산동읍 인덕리 280,b_b004042,B03,b004042,2209240004,NaN,NaN,2022-09-24,5.0
15406,NaN,3,구미에코랜드,경북 구미시 산동읍 인덕1길 195,경북 구미시 산동읍 인덕리 280,b_b011164,B03,b011164,2210270004,NaN,NaN,2022-10-27,4.0
37230,POI010000006W0H2H,6,에코랜드테마파크,제주특별자치도 제주시 조천읍 번영로 1278-169,제주특별자치도 제주시 조천읍 교래리 385-1,d_d000364,D01,d000364,2209070004,POI010000006W0H2H,에코랜드테마파크,2022-09-07,5.0
37537,POI010000006W0H2H,6,에코랜드테마파크,제주특별자치도 제주시 조천읍 번영로 1278-169,제주특별자치도 제주시 조천읍 교래리 385-1,d_d006122,D01,d006122,2210210005,POI010000006W0H2H,에코랜드테마파크,2022-10-21,4.0
38083,POI010000006W0H2H,6,에코랜드테마파크,제주특별자치도 제주시 조천읍 번영로 1278-169,제주특별자치도 제주시 조천읍 교래리 385-1,d_d006185,D01,d006185,2210140006,POI010000006W0H2H,에코랜드테마파크,2022-10-14,4.0
38476,NaN,1,제주 에코랜드,번영로 1278-169,NaN,d_d004163,NaN,NaN,2210150006,NaN,NaN,2022-10-15,5.0
38524,NaN,6,에코랜드 테마파크,NaN,NaN,d_d002165,D01,d002165,2210060002,NaN,NaN,2022-10-06,4.0
38977,NaN,6,에코랜드테마파크,제주특별자치도 제주시 조천읍 번영로 1278-169,제주특별자치도 제주시 조천읍 교래리 385-1,d_d001833,D01,d001833,2210090003,NaN,NaN,2022-10-09,4.0
39199,POI010000006W0H2H,6,에코랜드테마파크,제주특별자치도 제주시 조천읍 번영로 1278-169,제주특별자치도 제주시 조천읍 교래리 385-1,d_d011027,D01,d011027,2211050001,POI010000006W0H2H,에코랜드테마파크,2022-11-05,5.0
39252,NaN,6,에코랜드테마파크,제주특별자치도 제주시 조천읍 번영로 1278-169,제주특별자치도 제주시 조천읍 교래리 385-1,d_d004311,D01,d004311,2210100004,NaN,NaN,2022-10-10,4.0


In [189]:
test = travel_log[travel_log['VISIT_AREA_NM'].str.contains('카멜리아')]
test['VISIT_AREA_NM'].unique()

array(['구룡포 일본마을 동백꽃필무렵촬영지,카멜리아 카페', '카멜리아힐', '카멜리아힐(동백동산)'], dtype=object)

## POI 를 관광메타데이터로 통일하기

**작업 목표:**
- 한국 관광공사에서 제공하는 관광지 메타데이터인 travel_df와 외부 기업이 수집한 여행객 로그 데이터인 travel_log를 병합하여 관광지 추천 시스템을 만드는 것

**문제 상황:**
- 두 데이터프레임을 병합하기 위해서는 공통된 키(key)가 필요한데, travel_df의 contentid와 travel_log의 POI_ID가 이에 해당. 
    - 그러나 travel_log의 POI_ID 컬럼에는 null 값이 존재
    - travel_log의 장소명(VISIT_AREA_NM, POI_NM 등)이나 주소(ROAD_NM_ADDR, LOTNO_ADDR 등)가 travel_df의 해당 정보(title, addr1 등)와 일치하지 않는 경우가 있음
    - travel_log에서 동일한 장소를 나타내는 데이터라도 VISIT_AREA_NM이 서로 다르게 입력된 경우가 있음
    - 좌표 정보(X_COORD, Y_COORD)는 모바일 앱에서 수집된 사용자 위치 좌표로, travel_df의 관광지 좌표와 정확히 일치하지 않음.
    - 또한 반경 내에 여러 관광지가 존재할 수 있어 활용이 어려움

**해결 방안:**
- travel_log의 POI_ID가 null인 경우, 장소명이나 주소 정보를 이용하여 travel_df의 contentid와 매칭할 수 있을 것이란 가정
- 주소 정보를 기반으로 장소를 매칭하되, 한국의 주소 체계가 도로명 주소(ROAD_NM_ADDR)와 지번 주소(LOTNO_ADDR) 두 가지임을 고려
    - travel_df의 addr1이 도로명 주소 체계를 따르므로, travel_log의 ROAD_NM_ADDR과 비교하는 것이 적절함
- 주소 문자열에 대해 전처리(불필요한 공백, 특수문자 제거 등)를 수행한 후, 유사도 측정 알고리즘(Levenshtein distance, Jaro-Winkler distance 등)을 적용하여 유사도 점수를 계산
- 유사도 점수가 일정 임계값 이상인 경우, 해당 장소들을 동일한 장소로 간주하고 travel_log의 POI_ID에 travel_df의 contentid 값과 기타 관광지 관련 메타데이터를 할당해 데이터를 통일하는 작업 수행
- 매칭되지 않은 장소들에 대해서는 추가적인 전처리나 규칙 기반의 매칭 로직, 사용자 확인 요청 등의 방법을 사용할 수 있음

In [ ]:
from fuzzywuzzy import fuzz
from tqdm.auto import tqdm

def preprocess_address(address):
    # 주소가 null인 경우 빈 문자열로 변환, 그렇지 않으면 소문자로 변환하고 공백을 제거
    if pd.isna(address):
        return ""
    return address.lower().replace(" ", "")

def get_latest_chunk_number():
    # 현재 디렉터리의 파일 리스트 중 'matched_chunk_'로 시작하고 '.pkl'로 끝나는 파일 찾기
    pkl_files = [f for f in os.listdir() if f.endswith('.pkl') and f.startswith('matched_chunk_')]
    if not pkl_files:
        return 0  # 처리된 파일이 없으면 0 반환
    # 파일명에서 숫자만 추출하여 가장 큰 값 찾기
    max_chunk_number = max([int(f.split('_')[2].split('.')[0]) for f in pkl_files])
    return max_chunk_number

def match_places_in_chunks(travel_df, travel_log, threshold=80, chunk_size=1000):
    # 가장 최근 처리된 청크 번호 가져오기
    latest_chunk_number = get_latest_chunk_number()

    # 데이터프레임에 전처리된 주소 컬럼 추가
    travel_df['preprocessed_address'] = travel_df['addr1'].apply(preprocess_address)
    travel_log['preprocessed_address'] = travel_log['ROAD_NM_ADDR'].apply(preprocess_address)

    # 처리할 다음 청크의 시작 인덱스 계산
    start_index = (latest_chunk_number + 1) * chunk_size

    # tqdm을 사용하여 진행 상황을 시각적으로 표시
    for i in tqdm(range(start_index, len(travel_log), chunk_size), desc="전체 청크 처리"):
        chunk = travel_log[i:i+chunk_size]
        matched_chunk = []

        # 각 청크에 대해 반복 처리
        for _, row in tqdm(chunk.iterrows(), total=chunk.shape[0], desc="청크 내 항목 처리", leave=False):
            best_match = None
            best_score = 0
            # travel_df의 모든 행을 반복하며 유사도 점수 계산
            for _, df_row in travel_df.iterrows():
                score = fuzz.ratio(row['preprocessed_address'], df_row['preprocessed_address'])
                if score > best_score:
                    best_match = df_row['contentid']
                    best_score = score

            # 유사도 점수가 임계값 이상이면 최고 점수를 가진 contentid 할당
            if best_score >= threshold:
                row['POI_ID'] = best_match
            else:
                row['POI_ID'] = None
            matched_chunk.append(row)

        # 청크 데이터를 pickle 파일로 저장
        matched_chunk_df = pd.DataFrame(matched_chunk)
        latest_chunk_number += 1
        matched_chunk_df.to_pickle(f'matched_chunk_{latest_chunk_number}.pkl')

        # tqdm 인스턴스 초기화
        tqdm._instances.clear()

    # 모든 청크를 합쳐서 CSV 파일로 저장
    all_matched_chunks = [pd.read_pickle(f) for f in sorted(os.listdir()) if f.startswith('matched_chunk_')]
    matched_travel_log = pd.concat(all_matched_chunks, ignore_index=True)
    matched_travel_log.to_csv('converted_POI_travel_log.csv', encoding='utf-8-sig', index=False)

    return matched_travel_log

### 전체 데이터셋 작업 적용

In [ ]:
matched_travel_log = match_places_in_chunks(travel_df, travel_log, chunk_size=1000)

# 작업 결과 검증

- 함수 자체에 최종 파일을 저장하는 코드가 있었으므로 파일을 불러와서 재작업
- 관광메타 데이터의 POI_ID가 매칭된 경우
    - 올바르게 관광지 매칭이 되었는가
- 관광메타 데이터의 POI_ID가 매칭되지 않은 경우
    - 누락된 관광지중에 매칭이 되었어야할 장소는 없는가

In [309]:
matched_travel_log = pd.read_csv('converted_POI_travel_log.csv', encoding='utf-8-sig')
matched_travel_log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56909 entries, 0 to 56908
Data columns (total 34 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   TRAVEL_ID             56909 non-null  object 
 1   TRAVEL_NM             51143 non-null  object 
 2   TRAVELER_ID           51143 non-null  object 
 3   VISIT_AREA_ID         56909 non-null  int64  
 4   VISIT_AREA_NM         56909 non-null  object 
 5   VISIT_AREA_TYPE_CD    56909 non-null  int64  
 6   TRAVEL_YMD            56909 non-null  object 
 7   POI_ID                47975 non-null  float64
 8   POI_NM                26091 non-null  object 
 9   X_COORD               54342 non-null  object 
 10  Y_COORD               54342 non-null  object 
 11  SGG_CD                4285 non-null   float64
 12  ROAD_NM_CD            0 non-null      float64
 13  ROAD_NM_ADDR          36618 non-null  object 
 14  LOTNO_CD              964 non-null    object 
 15  LOTNO_ADDR         

## POI_ID가 매칭된 경우

### 장소가 올바르게 매칭 되었는가?

- 관광메타 정보의 contentid 와 전처리된 관광로그의 POI_ID가 일치하는 행들에 대해 아래의 작업 진행
    - 관광메타의 title, addr1 을 추출하고, 관광로그의 POI_ID, POI_NM, VISIT_AREA_NM, ROAD_NM_ADDR 을 추출해 데이터 프레임 생성
    - 생성한 df의 값들을 행별로 비교해서 서로 다른 값이 있는지 확인

In [310]:
def compare_addresses(addr1, road_nm_addr, sido_mapping):
    # 주소가 문자열이 아닌 경우 빈 문자열로 대체
    addr1 = str(addr1) if pd.notnull(addr1) else ""
    road_nm_addr = str(road_nm_addr) if pd.notnull(road_nm_addr) else ""

    # 주소 정규화
    normalized_addr1 = normalize_address(addr1, sido_mapping)
    normalized_road_nm_addr = normalize_address(road_nm_addr, sido_mapping)

    # 공백을 기준으로 주소 분할
    addr1_parts = normalized_addr1.split()
    road_nm_addr_parts = normalized_road_nm_addr.split()

    # 마지막 두 개의 값 추출
    addr1_last_two = ' '.join(addr1_parts[-2:])
    road_nm_addr_last_two = ' '.join(road_nm_addr_parts[-2:])

    # 추출한 값 비교
    if addr1_last_two == road_nm_addr_last_two:
        return '일치'
    else:
        return '불일치'

def compare_poi_data(travel_df, matched_travel_log):
    # POI_ID가 매칭된 데이터만 필터링
    matched_data = matched_travel_log[matched_travel_log['POI_ID'].notna()]

    # 관광메타 정보와 매칭된 관광로그 데이터 병합
    merged_data = pd.merge(travel_df, matched_data, left_on='contentid', right_on='POI_ID')

    # 비교를 위한 열 선택하여 새로운 데이터프레임 생성
    comparison_columns = ['contentid', 'title', 'addr1', 'POI_ID', 'POI_NM', 'VISIT_AREA_NM', 'ROAD_NM_ADDR']
    comparison_data = merged_data[comparison_columns].copy()

    # 시도명 축약형 매핑 딕셔너리
    sido_mapping = {
        '강원': ['강원도', '강원특별자치도'],
        '경기': ['경기도'],
        '경남': ['경상남도'],
        '경북': ['경상북도'],
        '전남': ['전라남도'],
        '전북': ['전라북도'],
        '충남': ['충청남도'],
        '충북': ['충청북도'],
        '제주': ['제주도', '제주특별자치도'],
        '서울': ['서울특별시', '서울시'],
        '부산': ['부산광역시', '부산시'],
        '대구': ['대구광역시', '대구시'],
        '인천': ['인천광역시', '인천시'],
        '광주': ['광주광역시', '광주시'],
        '대전': ['대전광역시', '대전시'],
        '울산': ['울산광역시', '울산시'],
        '세종': ['세종특별자치시', '세종시']
    }

    # 1차 필터링: 주소 비교 함수 적용
    comparison_data['1차_주소비교결과'] = comparison_data.apply(lambda x: compare_addresses(x['addr1'], x['ROAD_NM_ADDR'], sido_mapping), axis=1)

    # 1차 필터링 결과에 따른 주소 불일치 개수
    address_mismatch_count_1st = (comparison_data['1차_주소비교결과'] == '불일치').sum()
    print(f"1차 필터링 주소 불일치 개수: {address_mismatch_count_1st}")

    # 1차 필터링에서 불일치한 데이터만 추출
    mismatch_data = comparison_data[comparison_data['1차_주소비교결과'] == '불일치']

    # 2차 필터링: 장소명 비교
    mismatch_data['2차_장소명일치여부'] = mismatch_data.apply(lambda x: '일치' if x['title'] == x['POI_NM'] or x['title'] == x['VISIT_AREA_NM'] else '불일치', axis=1)

    # 2차 필터링 결과에 따른 주소 불일치 개수
    address_mismatch_count_2nd = (mismatch_data['2차_장소명일치여부'] == '불일치').sum()
    print(f"2차 필터링 주소 불일치 개수: {address_mismatch_count_2nd}")

    # 최종 결과 데이터프레임 생성
    result_data = pd.concat([comparison_data[comparison_data['1차_주소비교결과'] == '일치'], mismatch_data])

    return result_data

# POI 데이터 비교 함수 호출
result_df = compare_poi_data(travel_df, matched_travel_log)
print("\n비교 결과 데이터프레임:")
result_df

1차 필터링 주소 불일치 개수: 6960
2차 필터링 주소 불일치 개수: 6516

비교 결과 데이터프레임:


,contentid,title,addr1,POI_ID,POI_NM,VISIT_AREA_NM,ROAD_NM_ADDR,1차_주소비교결과,2차_장소명일치여부
0,126273,가계해수욕장,전라남도 진도군 고군면 신비의바닷길 47,126273.0,가계해변,가계해변,전남 진도군 고군면 신비의바닷길 47,일치,NaN
1,126273,가계해수욕장,전라남도 진도군 고군면 신비의바닷길 47,126273.0,가계해변,가계해변,전남 진도군 고군면 신비의바닷길 47,일치,NaN
2,129194,가나아트파크,경기도 양주시 장흥면 권율로 117,129194.0,장흥아트파크,가나아트파크,경기 양주시 장흥면 권율로 117,일치,NaN
3,129194,가나아트파크,경기도 양주시 장흥면 권율로 117,129194.0,NaN,가나아트파크 야외공연장,경기 양주시 장흥면 권율로 117,일치,NaN
4,129194,가나아트파크,경기도 양주시 장흥면 권율로 117,129194.0,장흥아트파크,가나아트파크 어린이체험관,경기 양주시 장흥면 권율로 117,일치,NaN
...,...,...,...,...,...,...,...,...,...
47914,1840837,[제주올레 19코스] 조천-김녕 올레,제주특별자치도 제주시 조천읍 조천리 1209-1,1840837.0,NaN,용천수탐방길,제주특별자치도 제주시 조천읍 조천1길 13-14,불일치,불일치
47915,1840837,[제주올레 19코스] 조천-김녕 올레,제주특별자치도 제주시 조천읍 조천리 1209-1,1840837.0,NaN,용천수탐방길,제주특별자치도 제주시 조천읍 조천1길 13-14,불일치,불일치
47916,1840837,[제주올레 19코스] 조천-김녕 올레,제주특별자치도 제주시 조천읍 조천리 1209-1,1840837.0,NaN,조천항,제주특별자치도 제주시 조천읍 조천북1길 11-13,불일치,불일치
47917,1839451,[제주올레 1코스] 시흥-광치기 올레,제주특별자치도 서귀포시 성산읍 시흥상동로 113,1839451.0,NaN,제주올레1코스 공식안내소,제주특별자치도 서귀포시 성산읍 시흥상동로53번길 88-46,불일치,불일치


- 결과로 나온 df를 봐도 주소가 일치하는 지역(경기도인데 경기로 표기하는 등) 임에도 불일치로 판정된 곳이 대다수임
    - 이를 고려해 마지막 2개의 주소 정보를 기반으로 비교하도록 해 수정 하니 불일치 개수가 확연히 줄어듬
    - 다만 아직도 나머지 데이터에 대해 확인해볼 필요가 있음
- POI_NM의 경우 VISIT_AREA_NM과 일치하지 않는 경우가 매우 많음을 확인
    - 대체로 title과 일치하는 데이터는 VISIT_AREA_NM 인 경우가 많음
- title에서 산책로, 둘레길, 코스로 되어 있는 경우는 여러지역을 거치는 관광지여서 획일화 하기 힘듬
    - 제거 필요

### 주소 불일치 데이터는 정말 불일치 데이터인가?

In [311]:
# '주소비교결과'가 '불일치'이고, 'title'에 '코스', '둘레길', '제#길'이 포함되지 않은 데이터 필터링
unmatched_addr = result_df[
    (result_df['2차_장소명일치여부'] == '불일치') &
    (~result_df['title'].str.contains('코스|둘레길|제\d+길'))
]

# 중복을 제거한 데이터프레임 생성
unmatched_addr_unique = unmatched_addr.drop_duplicates(subset='contentid')


# 결과 확인
display(unmatched_addr_unique.info())
display(unmatched_addr_unique)

<class 'pandas.core.frame.DataFrame'>
Index: 1074 entries, 13 to 47892
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   contentid      1074 non-null   int64  
 1   title          1074 non-null   object 
 2   addr1          1074 non-null   object 
 3   POI_ID         1074 non-null   float64
 4   POI_NM         572 non-null    object 
 5   VISIT_AREA_NM  1074 non-null   object 
 6   ROAD_NM_ADDR   1074 non-null   object 
 7   1차_주소비교결과      1074 non-null   object 
 8   2차_장소명일치여부     1074 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 83.9+ KB


None

,contentid,title,addr1,POI_ID,POI_NM,VISIT_AREA_NM,ROAD_NM_ADDR,1차_주소비교결과,2차_장소명일치여부
13,1884202,가메창(암메),제주특별자치도 제주시 한경면 청수로 13-3,1884202.0,고산성당,고산성당,제주특별자치도 제주시 한경면 칠전로 1,불일치,불일치
17,1884505,가문이오름(감은이오름),제주특별자치도 서귀포시 표선면 남조로 1487-73,1884505.0,NaN,보내다제주,제주특별자치도 서귀포시 표선면 토산중앙로 487-134,불일치,불일치
38,1885746,가세오름,제주특별자치도 서귀포시 표선면 녹산로 554,1885746.0,경기전사적지339호,블라제리조트 전기차충전소,제주특별자치도 서귀포시 표선면 녹산로 274,불일치,불일치
39,1885754,가시오름,제주특별자치도 서귀포시 대정읍 하모이삼로21번길 1,1885754.0,모슬포교회,모슬포교회,제주특별자치도 서귀포시 대정읍 하모이삼로15번길 25,불일치,불일치
53,127778,가천 다랭이마을,경남 남해군 남면 홍현리 남면로 679번길 21,127778.0,NaN,다랭이마을,경남 남해군 남면 남면로679번길 21,불일치,불일치
...,...,...,...,...,...,...,...,...,...
47776,2740985,힐링레저캠핑장&글램핑,경기도 가평군 가평읍 북한강변로 516,2740985.0,남이가든,옥연지 송해공원,경기도 가평군 가평읍 북한강변로 1016,불일치,불일치
47822,2609373,9.81 파크,제주특별자치도 제주시 애월읍 천덕로 880-24,2609373.0,NaN,부영농장,제주특별자치도 제주시 애월읍 천덕로 388,불일치,불일치
47882,2792780,J글램핑,경상남도 사천시 서포면 거북길 468-116,2792780.0,제이글램핑,니갤러리카페,경남 사천시 서포면 거북길 468-125,불일치,불일치
47884,752520,NFC(축구 국가대표팀 트레이닝 센터),경기도 파주시 탄현면 필승로 368,752520.0,NaN,통일동산두부마을,경기 파주시 탄현면 필승로 480,불일치,불일치


- 중복 contentid를 제외해도 약 1000여개의 데이터가 관찰될 정도로 낭비되는 데이터가 많음
    - 이중 실제로는 유사혹은 동일 장소임에도 필터링이 되지 못한 장소들이 다수 관찰 됨
- 보유 데이터가 그리 크지 않고, STARS 데이터의 클래스 불균형 이슈 때문에 최대한 데이터를 확보해야 함
- 최초 POI_ID 처리 로직의 유사도를 기준으로 추가 필터링을 시도해볼 필요 있음

In [312]:
import pandas as pd
from fuzzywuzzy import fuzz

def preprocess_text(text):
    if pd.isna(text):
        return ""
    return text.lower().replace(" ", "")

# 'title'과 'ROAD_NM_ADDR' 전처리
unmatched_addr_unique['preprocessed_title'] = unmatched_addr_unique['title'].apply(preprocess_text)
unmatched_addr_unique['preprocessed_address'] = unmatched_addr_unique['ROAD_NM_ADDR'].apply(preprocess_text)

# 'title'과 'ROAD_NM_ADDR' 유사도 비교
unmatched_addr_unique['similarity'] = unmatched_addr_unique.apply(lambda row: fuzz.ratio(row['preprocessed_title'], row['preprocessed_address']), axis=1)

# threshold가 특정 수치 이상인 행만 추출
unmatched_addr_df = unmatched_addr_unique[unmatched_addr_unique['similarity'] >= 30]

# 결과 출력
with pd.option_context('display.max_rows', None):  # 전체 행을 다 보게 설정
    display(len(unmatched_addr_df))
    display(unmatched_addr_df)

102

,contentid,title,addr1,POI_ID,POI_NM,VISIT_AREA_NM,ROAD_NM_ADDR,1차_주소비교결과,2차_장소명일치여부,preprocessed_title,preprocessed_address,similarity
75,921096,가평향교,경기도 가평군 가평읍 향교로 23-1,921096.0,가평초교,가평초등학교,경기 가평군 가평읍 향교로 23,불일치,불일치,가평향교,경기가평군가평읍향교로23,47
119,129024,갈두마을(땅끝마을),전라남도 해남군 송지면 땅끝마을길 82,129024.0,땅끝해양자연박물관,땅끝해양자연사박물관,전남 해남군 송지면 땅끝마을길 89,불일치,불일치,갈두마을(땅끝마을),전남해남군송지면땅끝마을길89,32
393,127110,강진 전라병영성,전라남도 강진군 병영면 병영성로 175,127110.0,전라병영성하멜기념관,하멜기념관,전남 강진군 병영면 병영성로 180,불일치,불일치,강진전라병영성,전남강진군병영면병영성로180,45
394,127111,강진영랑생가,전라남도 강진군 강진읍 영랑생가길 15,127111.0,세계모란공원,세계모란공원,전남 강진군 강진읍 영랑생가길 36,불일치,불일치,강진영랑생가,전남강진군강진읍영랑생가길36,57
545,538485,거제 구조라관광어촌마을,경상남도 거제시 일운면 구조라로4길 3-1,538485.0,NaN,구이조아라,경남 거제시 일운면 구조라로 44-1,불일치,불일치,거제구조라관광어촌마을,경남거제시일운면구조라로44-1,37
2629,126078,광안리해수욕장,부산광역시 수영구 광안해변로 219,126078.0,민락동씨랜드활어센타,민락씨랜드회센타,부산 수영구 광안해변로 299,불일치,불일치,광안리해수욕장,부산수영구광안해변로299,30
24366,2784747,나로도항,전라남도 고흥군 봉래면 나로도항길 128,2784747.0,나로도수산업협동조합,나로도수협,전남 고흥군 봉래면 나로도항길 135,불일치,불일치,나로도항,전남고흥군봉래면나로도항길135,40
25115,250094,담양 메타세쿼이아길,전라남도 담양군 담양읍 메타세쿼이아로 12,250094.0,NaN,메타세쿼이아 가로수길,전남 담양군 담양읍 메타세쿼이아로 25,불일치,불일치,담양메타세쿼이아길,전남담양군담양읍메타세쿼이아로25,62
25190,1954936,담양향교,전라남도 담양군 담양읍 향교길 19,1954936.0,NaN,명가,전남 담양군 담양읍 향교길 2,불일치,불일치,담양향교,전남담양군담양읍향교길2,50
25637,2783017,대천항유람선,충청남도 보령시 대천항중앙길 46,2783017.0,합기도검도,보령수협로컬푸드 바다듬,충남 보령시 대천항중앙길 76,불일치,불일치,대천항유람선,충남보령시대천항중앙길76,32


- similarity 점수를 60 이상으로 하면 일치 결과가 없음
- 60으로 했을때 4개의 유사 주소가 발견되고 50으로 했을때 15개, 40으로 했을 때 35개의 주소가 조회됨
- 40을 기준으로 지도앱, TourAPI의 DB에 조회하여 데이터를 최종 필터링
- 거리 기준으로 일정 threshold를 줘서 일괄 통일 작업을 고려했으나, POI의 카테고리느 중요한 정보이고, 불일치 데이터가 많지 않아서 최대한 자료 보존을 위해 수기 작업을 진행함

---

- **기록한 항목은 title(contentid, index) 순서**
    - 가평향교(contentid : 921096, index : 75) 
        - 총 1개의 데이터
        - 👉 POI_NM, VISIT_AREA_NM을 title로 통일하면 됨 
    - 담양 메타세쿼이아길(contentid : 250094, index : 25115) 
        - 👉 모든 contentid의 POI_NM, VISIT_AREA_NM을 title로 통일하면 됨
        - 48개의 데이터에 적용 가능
    - 대학로 (contentid : 126534, index: 25843) 
        - 👉 VISIT_AREA_NM이 맞는 케이스 (contentid : 126534 로 잘못 맵핑되어있음)
        - 총 4개의 데이터 
        - VISIT_AREA_NM가 `틴틴홀` 인 경우만 contentid : 126534 를 contentid : 3012760로 변경 해야함)
    - 문경새재 오픈세트장 (contentid : 2610293, index : 27686) 
        - 총 2개의 데이터
        - 👉 POI_NM, VISIT_AREA_NM을 title로 통일하면 됨        
    - 문경새재제1관문주흘관 (contentid : 2753508)
        - 총 11개의 데이터
        - 👉 POI_NM, VISIT_AREA_NM을 title로 통일하면 됨        
    - 보리나라 학원농장 (contentid : 125447)
        - 총 11개의 데이터
        - 👉 POI_NM, VISIT_AREA_NM을 title로 통일하면 됨        
    - 보문콜로세움 (contentid : 3000202)
        - 총 3개의 데이터
        - 👉 VISIT_AREA_NM이 맞는 케이스 (contentid : 2899295 로 맵핑되어야함)
    - 부산 송도해수욕장 (contentid : 126122, index : 28606)
        - 총 1개의 데이터
        - 👉 POI_NM, VISIT_AREA_NM을 title로 통일하면 됨        
    - 부산 송도해상케이블카 (contentid : 128829)
        - 총 10개의 데이터
         - 👉 VISIT_AREA_NM이 맞는 케이스 (contentid : 2504464 로 맵핑되어야함)
    - 사천진해변(사천뒷불해수욕장) (contentid : 585526)
        - 총 10개의 데이터
        - 👉 POI_NM, VISIT_AREA_NM을 title로 통일하면 됨       
    - 소령원 (contentid : 2569492, index : 31468)
        - 총 1개의 데이터
        - 👉 POI_NM, VISIT_AREA_NM을 title로 통일하면 됨       
    - 아산 외암마을 참판댁 (contentid : 1626699)
        - 총 21개의 데이터
        - 👉 VISIT_AREA_NM이 맞는 케이스 (contentid : 126001 로 맵핑되어야함)
    - 안동 소호헌 (contentid : 231940, index : 33804)
        - 총 1개의 데이터
        - 👉 POI_NM, VISIT_AREA_NM을 title로 통일하면 됨   
    - 예당관광지 (contentid : 128054)
        - 총 3개의 데이터
        - 👉 VISIT_AREA_NM이 맞는 케이스 (contentid : 3080441 로 맵핑되어야함)
    - 오대산국립공원 (contentid : 125591, index : 34951)
        - 총 1개의 데이터
        - 👉 POI_NM, VISIT_AREA_NM을 title로 통일하면 됨   
    - 와현모래숲해변 (contentid : 126578)
        - 총 1개의 데이터
        - 👉 POI_NM, VISIT_AREA_NM을 title로 통일하면 됨   
     - 유관순열사생가 (contentid : 2769773)
        - 총 3개의 데이터
        - 👉 POI_NM, VISIT_AREA_NM을 title로 통일하면 됨   
    - 장생포 고래바다여행선 (contentid : 769495)
        - 총 11개의 데이터
        - 👉 VISIT_AREA_NM이 맞는 케이스 (contentid : 130649 로 맵핑되어야함)
    - 정남진 편백숲 우드랜드 (contentid : 2033464)
        - 총 1개의 데이터
        - 👉 POI_NM, VISIT_AREA_NM을 title로 통일하면 됨   
    - 창녕 우포늪 (contentid : 126737)
        - 총 1개의 데이터
        - 👉 POI_NM, VISIT_AREA_NM을 title로 통일하면 됨   
    - 안성 남사당 공연장 (contentid : 130547)
        - 총 11개의 데이터 
        - 👉 VISIT_AREA_NM이 맞는 케이스 (contentid : 7213 으로 맵핑되어야함)
    - 당진 장고항 (contentid : 131815)
        - 총 7개의 데이터 
        - 👉 POI_NM, VISIT_AREA_NM을 title로 통일하면 됨   
---

- 위에 정리한 데이터의 경우 전처리를 통해 컬럼별로 값을 조정

#### 불일치 데이터 필터링

- unmatched_addr_df 의 출력결과에서 유사한 주소지로 보여지면 아래 2개의 함수를 써서 내용을 확인
- `find_matches_by_contentid` 를 사용해 주소치가 불일치한 데이터들(`unmatched`) 들 중 해당 `contentid` 를 가진 데이터가 몇개인지 확인
- 해당 함수의 출력결과를 통해 `contentid`를 기준으로 다른 데이터(`travel_log의 데이터`)를 변경할지 아님 반대로 `travel_log` 의 데이터(`POI_NM`, `VISIT_AREA_NM`) 를 기준으로 다른 데이터를 변경할지 확인할 수 있음

In [314]:
"""
contentid를 입력받아 unmatched 데이터프레임에서 일치하는 행의 개수와 해당 행들로 이루어진 데이터프레임을 출력하는 함수
"""
def find_matches_by_contentid(contentid):
    matches = unmatched_addr[unmatched_addr['contentid'] == contentid]
    match_count = len(matches)
    
    print(f"contentid가 {contentid}와 일치하는 행의 개수: {match_count}")
    display(matches[['contentid', 'title', 'addr1', 'POI_ID', 'POI_NM', 'VISIT_AREA_NM', 'ROAD_NM_ADDR']])

-  `travel_log` 의 데이터(`POI_NM`, `VISIT_AREA_NM`) 를 기준으로 다른 데이터를 변경해야 하는 경우 변경 대상을 확인하기 위해 관광메타 데이터(`travel_df`) 에서 해당 이름을 가진 관광지명을 검색해 `contentid` 를 확인
- 여기서 검색이 되는 데이터는 TourAPI에 있는데이터로서 공식 POI 정보라고 볼 수 있음

In [315]:
"""
키워드를 입력받아 travel_df 데이터프레임의 'title' 열에서 해당 키워드를 포함하는 | contentid에서 해당 값을 포함하는 행들을 출력하는 함수
"""
def find_matches_by_anything(keyword):
    # 입력값이 숫자인 경우 contentid 열에서 일치하는 값을 찾음
    if isinstance(keyword, int):
        matches = travel_df[travel_df['contentid'] == keyword]
    # 입력값이 문자열인 경우 title 열에서 해당 키워드를 포함하는 행을 찾음
    elif isinstance(keyword, str):
        matches = travel_df[travel_df['title'].str.contains(keyword, case=False, na=False)]
    else:
        print("입력값은 문자열 또는 숫자여야 합니다.")
        return

    # 결과 출력
    if not matches.empty:
        display(matches[['contentid', 'title', 'addr1']])
    else:
        print(f"'{keyword}'에 해당하는 결과가 없습니다.")

In [316]:
find_matches_by_contentid(130547)

contentid가 130547와 일치하는 행의 개수: 11


,contentid,title,addr1,POI_ID,POI_NM,VISIT_AREA_NM,ROAD_NM_ADDR
44508,130547,안성 남사당 공연장,경기도 안성시 보개면 남사당로 198-2,130547.0,안성맞춤랜드,안성맞춤랜드,경기 안성시 보개면 남사당로 198
44509,130547,안성 남사당 공연장,경기도 안성시 보개면 남사당로 198-2,130547.0,안성맞춤랜드,안성맞춤랜드,경기 안성시 보개면 남사당로 198
44510,130547,안성 남사당 공연장,경기도 안성시 보개면 남사당로 198-2,130547.0,안성맞춤랜드,안성맞춤랜드,경기 안성시 보개면 남사당로 198
44511,130547,안성 남사당 공연장,경기도 안성시 보개면 남사당로 198-2,130547.0,NaN,안성맞춤랜드,경기 안성시 보개면 남사당로 198
44512,130547,안성 남사당 공연장,경기도 안성시 보개면 남사당로 198-2,130547.0,안성맞춤랜드,안성맞춤랜드 숲속공연장,경기 안성시 보개면 남사당로 198
44513,130547,안성 남사당 공연장,경기도 안성시 보개면 남사당로 198-2,130547.0,NaN,안성맞춤 남사당바우덕이축제,경기 안성시 보개면 남사당로 198
44514,130547,안성 남사당 공연장,경기도 안성시 보개면 남사당로 198-2,130547.0,안성맞춤랜드,안성맞춤 남사당바우덕이축제,경기 안성시 보개면 남사당로 198
44515,130547,안성 남사당 공연장,경기도 안성시 보개면 남사당로 198-2,130547.0,MG새마을금고제주연수원,안성맞춤 남사당바우덕이축제,경기 안성시 보개면 남사당로 198
44516,130547,안성 남사당 공연장,경기도 안성시 보개면 남사당로 198-2,130547.0,안성맞춤랜드,안성맞춤랜드,경기 안성시 보개면 남사당로 198
44517,130547,안성 남사당 공연장,경기도 안성시 보개면 남사당로 198-2,130547.0,안성맞춤랜드,안성맞춤랜드 잔디공원,경기 안성시 보개면 남사당로 198


In [317]:
find_matches_by_anything('안성맞춤랜드')

,contentid,title,addr1
7213,2525563,안성맞춤랜드,경기도 안성시 보개면 남사당로 196-31


- 부가적으로 장소명이 매우 유사해보이거나 공식명칭과 일부 다른 경우 등 유사 장소라 생각되는 곳들은
    - 카카오맵에 검색하고 두 주소(`addr1` 과 `ROAD_NM_ADDR`) 를 검색해 길찾기를 해서 100m 이내에 위치한 경우 이면서 (보통 같은 관광지인데 관광지 내 여러 시설이 있는 경우)
    - 두 장소의 카테고리유형이 동일한 경우만 같은 장소로 취급 

### POI_ID 매칭된 데이터중 오류 데이터 수정 전처리

In [318]:
merged_df = pd.merge(matched_travel_log, travel_df, left_on = 'POI_ID', right_on = 'contentid')

In [319]:
# POI_NM 및 VISIT_AREA_NM을 title로 통일하고, addr1을 ROAD_NM_ADDR로 변경하는 작업을 수행합니다.


# POI_NM 및 VISIT_AREA_NM을 title로 통일해야 하는 contentid 목록
title_unification_ids = [
    921096,  # 가평향교
    250094,  # 담양 메타세쿼이아길
    2610293,  # 문경새재 오픈세트장
    2753508,  # 문경새재제1관문주흘관
    125447,   # 보리나라 학원농장
    126122,   # 부산 송도해수욕장
    585526,   # 사천진해변(사천뒷불해수욕장)
    2569492,  # 소령원
    231940,   # 안동 소호헌
    125591,   # 오대산국립공원
    126578,   # 와현모래숲해변
    2769773,  # 유관순열사생가
    2033464,  # 정남진 편백숲 우드랜드
    126737,   # 창녕 우포늪
    131815    # 당진 장고항
]

# contentid에 따른 VISIT_AREA_NM 조건 변경 목록
visit_area_nm_condition_changes = {
    126534: 3012760,  # 대학로 -> 틴틴홀
    3000202: 2899295,  # 보문콜로세움
    128829: 2504464,   # 부산 송도해상케이블카
    1626699: 126001,   # 아산 외암마을 참판댁
    128054: 3080441,   # 예당관광지
    769495: 130649,    # 장생포 고래바다여행선
    130547: 2525563    # 안성 남사당 공연장 -> 안성맞춤랜드
}

# title_unification_ids에 대한 작업
for contentid in title_unification_ids:
    # title로 POI_NM 및 VISIT_AREA_NM 통일
    title_value = merged_df.loc[merged_df['contentid'] == contentid, 'title'].values[0]
    merged_df.loc[merged_df['contentid'] == contentid, ['POI_NM', 'VISIT_AREA_NM']] = title_value
    
    # addr1 값을 ROAD_NM_ADDR로 업데이트
    addr1_value = merged_df.loc[merged_df['contentid'] == contentid, 'addr1'].values[0]
    merged_df.loc[merged_df['contentid'] == contentid, 'ROAD_NM_ADDR'] = addr1_value

# visit_area_nm_condition_changes에 대한 작업 수정
for original_contentid, new_contentid in visit_area_nm_condition_changes.items():
    # 변경할 대상 컬럼의 값들을 travel_df에서 가져오기
    if not travel_df[travel_df['contentid'] == new_contentid].empty:
        title_value = travel_df.loc[travel_df['contentid'] == new_contentid, 'title'].values[0]
        addr1_value = travel_df.loc[travel_df['contentid'] == new_contentid, 'addr1'].values[0]
    else:
        print(f"contentid {new_contentid}에 해당하는 데이터가 travel_df에 없습니다.")
        continue
    
    # merged_df에서 original_contentid를 검색하여 변경할 대상이 되는 행들 추출
    if original_contentid == 126534 and new_contentid == 3012760:
        # '틴틴홀'인 경우만 특별 처리
        condition = (merged_df['contentid'] == original_contentid) & (merged_df['VISIT_AREA_NM'] == '틴틴홀')
        merged_df.loc[condition, 'contentid'] = new_contentid
        merged_df.loc[condition, ['POI_NM', 'VISIT_AREA_NM', 'ROAD_NM_ADDR']] = [title_value, title_value, addr1_value]
    else:
        # 일반적인 경우 처리
        condition = merged_df['contentid'] == original_contentid
        merged_df.loc[condition, 'contentid'] = new_contentid
        merged_df.loc[condition, ['POI_NM', 'VISIT_AREA_NM', 'ROAD_NM_ADDR']] = [title_value, title_value, addr1_value]



# 변경 사항 확인
merged_df.head()

,TRAVEL_ID,TRAVEL_NM,TRAVELER_ID,VISIT_AREA_ID,VISIT_AREA_NM,VISIT_AREA_TYPE_CD,TRAVEL_YMD,POI_ID,POI_NM,X_COORD,...,cpyrhtDivCd,mapx,mapy,mlevel,modifiedtime,sigungucode,tel,title,zipcode,standardized_region
0,a_a003598,A03,a003598,2210010008,포천아트밸리,3,2022-10-01,2464201.0,포천아트밸리,127.2372456,...,Type3,127.236518,37.923501,6.0,20240311163744,29.0,NaN,포천아트밸리 (한탄강 유네스코 세계지질공원),11139,경기도
1,a_a005299,A03,a005299,2210080003,포천아트밸리,1,2022-10-08,2464201.0,포천아트밸리,127.2372456,...,Type3,127.236518,37.923501,6.0,20240311163744,29.0,NaN,포천아트밸리 (한탄강 유네스코 세계지질공원),11139,경기도
2,a_a008658,A03,a008658,2210290003,포천아트밸리,1,2022-10-29,2464201.0,포천아트밸리,127.2372456,...,Type3,127.236518,37.923501,6.0,20240311163744,29.0,NaN,포천아트밸리 (한탄강 유네스코 세계지질공원),11139,경기도
3,a_a002746,A03,a002746,2209250002,포천아트밸리,1,2022-09-25,2464201.0,포천아트밸리,127.2372456,...,Type3,127.236518,37.923501,6.0,20240311163744,29.0,NaN,포천아트밸리 (한탄강 유네스코 세계지질공원),11139,경기도
4,a_a002888,A03,a002888,2209260004,포천아트밸리,1,2022-09-26,2464201.0,포천아트밸리,127.2372456,...,Type3,127.236518,37.923501,6.0,20240311163744,29.0,NaN,포천아트밸리 (한탄강 유네스코 세계지질공원),11139,경기도


#### 검증

In [320]:
def find_matches_by_contentid2(contentid):
    matches = merged_df[merged_df['contentid'] == contentid]
    match_count = len(matches)
    
    print(f"contentid가 {contentid}와 일치하는 행의 개수: {match_count}")
    display(matches[['contentid', 'title', 'addr1', 'POI_ID', 'POI_NM', 'VISIT_AREA_NM', 'ROAD_NM_ADDR']])

In [321]:
find_matches_by_contentid2(2504464)

contentid가 2504464와 일치하는 행의 개수: 27


,contentid,title,addr1,POI_ID,POI_NM,VISIT_AREA_NM,ROAD_NM_ADDR
28905,2504464,부산 암남공원,부산광역시 서구 암남공원로 185,128829.0,부산 송도해상케이블카,부산 송도해상케이블카,부산광역시 서구 송도해변로 171
28906,2504464,부산 암남공원,부산광역시 서구 암남공원로 185,128829.0,부산 송도해상케이블카,부산 송도해상케이블카,부산광역시 서구 송도해변로 171
28907,2504464,부산 암남공원,부산광역시 서구 암남공원로 185,128829.0,부산 송도해상케이블카,부산 송도해상케이블카,부산광역시 서구 송도해변로 171
28908,2504464,부산 암남공원,부산광역시 서구 암남공원로 185,128829.0,부산 송도해상케이블카,부산 송도해상케이블카,부산광역시 서구 송도해변로 171
28909,2504464,부산 암남공원,부산광역시 서구 암남공원로 185,128829.0,부산 송도해상케이블카,부산 송도해상케이블카,부산광역시 서구 송도해변로 171
28910,2504464,부산 암남공원,부산광역시 서구 암남공원로 185,128829.0,부산 송도해상케이블카,부산 송도해상케이블카,부산광역시 서구 송도해변로 171
28911,2504464,부산 암남공원,부산광역시 서구 암남공원로 185,128829.0,부산 송도해상케이블카,부산 송도해상케이블카,부산광역시 서구 송도해변로 171
28912,2504464,부산 암남공원,부산광역시 서구 암남공원로 185,128829.0,부산 송도해상케이블카,부산 송도해상케이블카,부산광역시 서구 송도해변로 171
28913,2504464,부산 암남공원,부산광역시 서구 암남공원로 185,128829.0,부산 송도해상케이블카,부산 송도해상케이블카,부산광역시 서구 송도해변로 171
28914,2504464,부산 암남공원,부산광역시 서구 암남공원로 185,128829.0,부산 송도해상케이블카,부산 송도해상케이블카,부산광역시 서구 송도해변로 171


In [322]:
find_matches_by_keyword(3012760)

,addr1,addr2,areacode,booktour,cat1,cat2,cat3,contentid,contenttypeid,createdtime,...,cpyrhtDivCd,mapx,mapy,mlevel,modifiedtime,sigungucode,tel,title,zipcode,standardized_region
15345,서울특별시 종로구 대학로10길 24 (동숭동),NaN,1.0,NaN,A02,A0206,A02060600,3012760,14,20230920113924,...,Type3,127.003568,37.581653,6.0,20230920113940,23.0,NaN,틴틴홀,03086,서울특별시


- 지정한데로 올바르게 매칭이나 변경이 잘 된 것으로 확인된다.
- 해당 데이터를 3가지 데이터로 개별 저장
    - travel_df : 관광 메타 데이터
    - travel_log : 여행기록 데이터
    - merged_df : 추천모델용 전체 데이터

In [328]:
merged_df.columns

Index(['TRAVEL_ID', 'TRAVEL_NM', 'TRAVELER_ID', 'VISIT_AREA_ID',
       'VISIT_AREA_NM', 'VISIT_AREA_TYPE_CD', 'TRAVEL_YMD', 'POI_ID', 'POI_NM',
       'X_COORD', 'Y_COORD', 'SGG_CD', 'ROAD_NM_CD', 'ROAD_NM_ADDR',
       'LOTNO_CD', 'LOTNO_ADDR', 'REVISIT_INTENTION', 'RCMDTN_INTENTION',
       'TRAVEL_MISSION_CHECK', 'GENDER', 'AGE_GRP', 'TRAVEL_STYL_1',
       'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5',
       'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8', 'TRAVEL_MOTIVE_1',
       'TRAVEL_MOTIVE_2', 'TRAVEL_MOTIVE_3', 'STARS', 'preprocessed_address',
       'addr1', 'addr2', 'areacode', 'booktour', 'cat1', 'cat2', 'cat3',
       'contentid', 'contenttypeid', 'createdtime', 'firstimage',
       'firstimage2', 'cpyrhtDivCd', 'mapx', 'mapy', 'mlevel', 'modifiedtime',
       'sigungucode', 'tel', 'title', 'zipcode', 'standardized_region'],
      dtype='object')

In [330]:
merged_df[['TRAVEL_ID', 'TRAVEL_NM', 'TRAVELER_ID', 'VISIT_AREA_ID',
       'VISIT_AREA_TYPE_CD', 'TRAVEL_YMD', 'POI_ID', 'POI_NM',
       'X_COORD', 'Y_COORD', 'SGG_CD', 'ROAD_NM_CD', 'ROAD_NM_ADDR',
       'LOTNO_CD', 'LOTNO_ADDR', 'REVISIT_INTENTION', 'RCMDTN_INTENTION',
        'GENDER', 'AGE_GRP', 'TRAVEL_STYL_1','TRAVEL_STYL_2', 'TRAVEL_STYL_3',
       'TRAVEL_STYL_4', 'TRAVEL_STYL_5','TRAVEL_STYL_6', 'TRAVEL_STYL_7', 
       'TRAVEL_STYL_8', 'STARS']]

,TRAVEL_ID,TRAVEL_NM,TRAVELER_ID,VISIT_AREA_ID,VISIT_AREA_TYPE_CD,TRAVEL_YMD,POI_ID,POI_NM,X_COORD,Y_COORD,...,AGE_GRP,TRAVEL_STYL_1,TRAVEL_STYL_2,TRAVEL_STYL_3,TRAVEL_STYL_4,TRAVEL_STYL_5,TRAVEL_STYL_6,TRAVEL_STYL_7,TRAVEL_STYL_8,STARS
0,a_a003598,A03,a003598,2210010008,3,2022-10-01,2464201.0,포천아트밸리,127.2372456,37.923174,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
1,a_a005299,A03,a005299,2210080003,1,2022-10-08,2464201.0,포천아트밸리,127.2372456,37.923174,...,30.0,5.0,4.0,5.0,5.0,4.0,5.0,5.0,5.0,4.0
2,a_a008658,A03,a008658,2210290003,1,2022-10-29,2464201.0,포천아트밸리,127.2372456,37.923174,...,40.0,2.0,1.0,3.0,5.0,4.0,6.0,3.0,6.0,3.0
3,a_a002746,A03,a002746,2209250002,1,2022-09-25,2464201.0,포천아트밸리,127.2372456,37.923174,...,20.0,5.0,1.0,3.0,6.0,4.0,4.0,4.0,4.0,4.0
4,a_a002888,A03,a002888,2209260004,1,2022-09-26,2464201.0,포천아트밸리,127.2372456,37.923174,...,30.0,2.0,3.0,5.0,5.0,6.0,6.0,6.0,2.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47970,a_a001366,A03,a001366,2208260002,2,2022-08-26,2554274.0,NaN,127.581096,37.547634,...,20.0,2.0,1.0,1.0,5.0,6.0,6.0,2.0,3.0,3.0
47971,a_a017936,A01,a017936,2211120004,1,2022-11-12,2675098.0,푸른수목원,126.825413,37.483983,...,50.0,2.0,4.0,2.0,4.0,2.0,2.0,6.0,2.0,4.0
47972,a_a004662,A03,a004662,2209300003,1,2022-09-30,2764307.0,온천공원,127.45248,37.281425,...,20.0,4.0,5.0,5.0,6.0,4.0,5.0,5.0,5.0,4.0
47973,a_a014792,A01,a014792,2211040009,3,2022-11-04,130577.0,메이플치과,126.984942,37.572557,...,40.0,1.0,4.0,4.0,3.0,4.0,3.0,5.0,3.0,4.0


In [331]:
filtered_travel_log = merged_df[['TRAVEL_ID', 'TRAVEL_NM', 'TRAVELER_ID', 'VISIT_AREA_ID',
       'VISIT_AREA_TYPE_CD', 'TRAVEL_YMD', 'POI_ID', 'POI_NM',
       'X_COORD', 'Y_COORD', 'SGG_CD', 'ROAD_NM_CD', 'ROAD_NM_ADDR',
       'LOTNO_CD', 'LOTNO_ADDR', 'REVISIT_INTENTION', 'RCMDTN_INTENTION',
        'GENDER', 'AGE_GRP', 'TRAVEL_STYL_1','TRAVEL_STYL_2', 'TRAVEL_STYL_3',
       'TRAVEL_STYL_4', 'TRAVEL_STYL_5','TRAVEL_STYL_6', 'TRAVEL_STYL_7', 
       'TRAVEL_STYL_8', 'STARS']]

In [332]:
filtered_travel_log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47975 entries, 0 to 47974
Data columns (total 28 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   TRAVEL_ID           47975 non-null  object 
 1   TRAVEL_NM           43164 non-null  object 
 2   TRAVELER_ID         43164 non-null  object 
 3   VISIT_AREA_ID       47975 non-null  int64  
 4   VISIT_AREA_TYPE_CD  47975 non-null  int64  
 5   TRAVEL_YMD          47975 non-null  object 
 6   POI_ID              47975 non-null  float64
 7   POI_NM              21587 non-null  object 
 8   X_COORD             45952 non-null  object 
 9   Y_COORD             45952 non-null  object 
 10  SGG_CD              3323 non-null   float64
 11  ROAD_NM_CD          0 non-null      float64
 12  ROAD_NM_ADDR        27684 non-null  object 
 13  LOTNO_CD            727 non-null    object 
 14  LOTNO_ADDR          44685 non-null  object 
 15  REVISIT_INTENTION   47938 non-null  float64
 16  RCMD

In [333]:
filtered_travel_log.to_csv('../data/travel_log/travel_log_final.csv', encoding='utf-8-sig', index=False)

In [327]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47975 entries, 0 to 47974
Data columns (total 56 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   TRAVEL_ID             47975 non-null  object 
 1   TRAVEL_NM             43164 non-null  object 
 2   TRAVELER_ID           43164 non-null  object 
 3   VISIT_AREA_ID         47975 non-null  int64  
 4   VISIT_AREA_NM         47975 non-null  object 
 5   VISIT_AREA_TYPE_CD    47975 non-null  int64  
 6   TRAVEL_YMD            47975 non-null  object 
 7   POI_ID                47975 non-null  float64
 8   POI_NM                21587 non-null  object 
 9   X_COORD               45952 non-null  object 
 10  Y_COORD               45952 non-null  object 
 11  SGG_CD                3323 non-null   float64
 12  ROAD_NM_CD            0 non-null      float64
 13  ROAD_NM_ADDR          27684 non-null  object 
 14  LOTNO_CD              727 non-null    object 
 15  LOTNO_ADDR         

## POI_ID가 매칭 되지 않은 경우

- POI_ID가 매칭되지 않았다는것은 기본적으로 `addr1` 과 `ROAD_NM_ADDR` 의 유사도가 80이하였다는 것으로 주소지에 상당한 차이가 있다는 의미
- 다만, 여행로그에 기록된 장소명에 대해서는 검증하지 않았으므로 일종의 2차 통일 작업이 필요하다 볼 수 있음
- 따라서 POI_ID가 null인 데이터들에 대해 명칭을 비교해서 확보 가능한 데이터가 있는지 검증할 예정
- 전체 여행로그 데이터가 5.6만여개에 불과하기 때문에 POI_ID가 매칭되지 않은 약 9천개의 데이터는 상당히 큰 볼륨이므로 최대한 데이터를 유지하기 위한 시도임

In [323]:
matched_travel_log['POI_ID'].isnull().sum()

8934

In [324]:
unmatched_df = matched_travel_log[matched_travel_log['POI_ID'].isnull()].info()

<class 'pandas.core.frame.DataFrame'>
Index: 8934 entries, 2 to 56894
Data columns (total 34 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   TRAVEL_ID             8934 non-null   object 
 1   TRAVEL_NM             7979 non-null   object 
 2   TRAVELER_ID           7979 non-null   object 
 3   VISIT_AREA_ID         8934 non-null   int64  
 4   VISIT_AREA_NM         8934 non-null   object 
 5   VISIT_AREA_TYPE_CD    8934 non-null   int64  
 6   TRAVEL_YMD            8934 non-null   object 
 7   POI_ID                0 non-null      float64
 8   POI_NM                4633 non-null   object 
 9   X_COORD               8390 non-null   object 
 10  Y_COORD               8390 non-null   object 
 11  SGG_CD                962 non-null    float64
 12  ROAD_NM_CD            0 non-null      float64
 13  ROAD_NM_ADDR          8934 non-null   object 
 14  LOTNO_CD              237 non-null    object 
 15  LOTNO_ADDR            804

In [325]:
from tqdm import tqdm

# POI_ID가 null인 데이터만 추출
null_poi_id_data = matched_travel_log[matched_travel_log['POI_ID'].isnull()]

# travel_df의 'title'과 null_poi_id_data의 'POI_NM', 'VISIT_AREA_NM'을 각각 비교하여 유사도 계산
similarity_scores_poi_nm = []
similarity_scores_visit_area_nm = []
content_ids = []

for idx, row in tqdm(null_poi_id_data.iterrows(), total=len(null_poi_id_data), desc="Calculating similarities"):
    poi_nm = str(row['POI_NM']) if pd.notnull(row['POI_NM']) else ''
    visit_area_nm = str(row['VISIT_AREA_NM']) if pd.notnull(row['VISIT_AREA_NM']) else ''
    
    max_similarity_poi_nm = 0
    max_similarity_visit_area_nm = 0
    max_content_id = None
    
    for _, travel_row in travel_df.iterrows():
        title = str(travel_row['title'])
        
        similarity_poi_nm = fuzz.ratio(title, poi_nm)
        similarity_visit_area_nm = fuzz.ratio(title, visit_area_nm)
        
        if similarity_poi_nm > max_similarity_poi_nm or similarity_visit_area_nm > max_similarity_visit_area_nm:
            max_similarity_poi_nm = similarity_poi_nm
            max_similarity_visit_area_nm = similarity_visit_area_nm
            max_content_id = travel_row['contentid']
    
    similarity_scores_poi_nm.append(max_similarity_poi_nm)
    similarity_scores_visit_area_nm.append(max_similarity_visit_area_nm)
    content_ids.append(max_content_id)

# null_poi_id_data에 유사도 점수와 content_id 열 추가
null_poi_id_data['similarity_score_poi_nm'] = similarity_scores_poi_nm
null_poi_id_data['similarity_score_visit_area_nm'] = similarity_scores_visit_area_nm
null_poi_id_data['matched_content_id'] = content_ids

# 유사도가 90 이상인 경우 POI_ID에 contentid 할당
for idx, row in tqdm(null_poi_id_data.iterrows(), total=len(null_poi_id_data), desc="Assigning contentid to POI_ID"):
    if row['similarity_score_poi_nm'] >= 90 or row['similarity_score_visit_area_nm'] >= 90:
        matched_travel_log.at[idx, 'POI_ID'] = row['matched_content_id']

# 유사도가 90 이상인 데이터만 필터링
filtered_data = null_poi_id_data[(null_poi_id_data['similarity_score_poi_nm'] >= 80) | (null_poi_id_data['similarity_score_visit_area_nm'] >= 90)]

# 결과 출력
display("Filtered Data:")
display(filtered_data)
display("\nUpdated matched_travel_log DataFrame:")
display(matched_travel_log)

Calculating similarities:   0%|                                                     | 6/8934 [00:07<3:01:44,  1.22s/it]


KeyboardInterrupt: 

In [300]:
filtered_data['POI_ID'].isnull().sum()

889

In [301]:
# matched_travel_log의 컬럼과 travel_df를 merge
merged_df = pd.merge(filtered_data, travel_df, left_on='POI_ID', right_on='contentid', how='inner')

# 결과 확인
display(merged_df.head())

,TRAVEL_ID,TRAVEL_NM,TRAVELER_ID,VISIT_AREA_ID,VISIT_AREA_NM,VISIT_AREA_TYPE_CD,TRAVEL_YMD,POI_ID,POI_NM,X_COORD,...,cpyrhtDivCd,mapx,mapy,mlevel,modifiedtime,sigungucode,tel,title,zipcode,standardized_region


### 결론

- 여러가지 시도를 해보았으나 제대로 맵핑이 되지 않음
- 특히 로직의 오류인듯 하나 POI_ID에 contentid가 제대로 맵핑이 되지 않음